## Advent of code 2022 day 11-20
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv-pypy environment for pypy 3.9
# to activate it from a git bash shell: source .venv-pypy/Scripts/activate
# to generate its requirements: pip freeze > .venv-pypy-requirements.txt

import collections
import itertools
import functools
import re
import copy
import math
import sys
import time
import json
import heapq
import bisect
import random
import sortedcontainers
#import cProfile

In [ ]:
# utility functions and version check

def get_line_groups(lines, nostrip=False):
    '''return list of lists of lines, each separated by empty lines, ignores empty lines from start and end,
    by default also strips all lines (if nostrip is set only strips empty lines)'''
    lines=list(lines)
    lines.append('') # add terminator
    res=[]
    group=[]
    for line in lines:
        line_str=line.strip()
        if nostrip==False or len(line_str)<1:
            line=line_str
        if len(line)>0:
            group.append(line)
        elif len(group)>0: # close group
            res.append(group)
            group=[]
    return res

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def exit():
    raise StopExecution()
    
print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}') # supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

In [ ]:
# 2022 day 17
# mv ~/Downloads/input* data_src/2022-day-17-input.txt
# big input file looks like: single line of movements of over 10k chars
# idea: part 1 parse movements and blocks, then simulate, painting walls
#  and floor as well to ease collision detection

sample2='''
>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>
'''

blocks='''
####

.#.
###
.#.

..#
..#
###

#
#
#
#

##
##
'''

def draw_floor(board, bwidth):
    for x in range(0, bwidth+2):
        board[ (x,0) ]='-'
    board['miny']=0

def draw_walls(board, bwidth, startclearing):
    # miny-startclearing-5
    for y in range(board['miny']-startclearing-5, 0):
        if (0,y) in board:
            break
        board[ (0,y) ]='|'
        board[ (bwidth+1,y) ]='|'

def print_board(board):
    y_vals=[tup[1] for tup in board.keys() if tup!='miny' ]
    x_vals=[tup[0] for tup in board.keys() if tup!='miny' ]
    print(f'board miny={board["miny"]}')
    res=[]
    for y in range(min(y_vals), max(y_vals)+1):
        row=''
        for x in range(min(x_vals), max(x_vals)+1):
            c=board.get( (x, y) , ' ')
            row+=c
        print(row)

def draw_block(board, blocklines, x0, y0, real=False):
    for y, row in enumerate(blocklines):
        for x, c in enumerate(row):
            if c!='#':
                continue
            if (x0+x, y0+y) in board:
                return False
            if real:
                board[(x0+x, y0+y)]=c
    return True

def drop_block(board, blocklines, moves, movei, startx, startclearing):
    # determine x,y of top left of block
    x=startx
    y=board['miny']-startclearing-len(blocklines)
    # check-draw / assert
    assert draw_block(board, blocklines, x, y)
    while True:
        # move left/right / check-draw if fails move back
        movec=moves[movei]
        movei=(movei+1) % len(moves)
        oldx=x
        if movec=='<':
            x-=1
        elif movec=='>':
            x+=1
        else:
            assert False
        if not draw_block(board, blocklines, x, y):
            x=oldx
        # move down / check-draw if fails move back and stop
        y+=1
        if not draw_block(board, blocklines, x, y):
            y-=1
            break
    # check-draw for real / assert
    assert draw_block(board, blocklines, x, y, real=True)
    # update miny
    board['miny']=min(y, board['miny'])
    return movei

sample1=open('data_src/2022-day-17-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
assert len(lines)==1
moves=lines[0]
bgroups=get_line_groups(blocks.splitlines(), nostrip=True)
bwidth=7
startx=3
startclearing=3
board={} # maps x,y to char, 'miny' to minimum y of rocks
# floor is at 0,0 and to the right (positive x),
# left wall is at 0,0 and up (negative y)
# right wall is at bwidth+1,0 and up (negative y)
draw_floor(board, bwidth)
groupi=0
movei=0
for _ in range(2022):
    draw_walls(board, bwidth, startclearing)
    movei=drop_block(board, bgroups[groupi], moves, movei, startx, startclearing)
    #print_board(board)
    groupi=(groupi+1) % len(bgroups)
height=abs(board['miny'])
print(f'part 1: {height}')

# part 1: 3067

In [ ]:
# part 2
# keep drawing until we have an instance of repeated groupi and movei,
# then we can skip ahead and only have to draw remaining turns

board={}
draw_floor(board, bwidth)
groupi=0
movei=0
rocks=0
tturns=1000000000000
gd={} # maps movei to (rocks, height) when groupi==0
repcount=0 # skipping the first few repeats to allow it to settle down
fakeheight=0
while rocks<tturns:
    if groupi==0 and fakeheight==0:
        newval=(rocks, abs(board['miny']))
        if movei in gd:
            oldval=gd[movei]
            repcount+=1
            if repcount==10:
                perrocks=newval[0]-oldval[0]
                perheight=newval[1]-oldval[1]
                print(f'{perrocks} rocks gives {perheight} height')
                remturns=tturns-rocks
                skiptimes=remturns//perrocks-2
                rocks+=skiptimes*perrocks # skipping ahead
                fakeheight=skiptimes*perheight
        gd[movei]= newval
    draw_walls(board, bwidth, startclearing)
    movei=drop_block(board, bgroups[groupi], moves, movei, startx, startclearing)
    #print_board(board)
    groupi=(groupi+1) % len(bgroups)
    rocks+=1
# sample2: 35 rocks gives 53 height
# sample1: 1705 rocks gives 2582 height
height=abs(board['miny'])
print(f'part 2: {height+fakeheight}')

# part 2: 1514369501484

In [ ]:
# 2022 day 16
# mv ~/Downloads/input* data_src/2022-day-16-input.txt
# big input file looks like: 58 valves of which 15 have non-zero flow
# idea: part 1 parse using re, then BFS based on a state consisting of current position
#  and set of opened valves

sample2='''
Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II
'''

def do_steps0(data, startpos):
    '''per position, how fast can you reach any other position?
    start at startpos, BFS, data maps position to (flow rate, [tunnel destinations])'''
    # BFS
    todos=set()
    todos.add(startpos)
    reached={} # maps position to min steps to reach
    reached[startpos]=0
    while len(todos)>0:
        pos=todos.pop()
        tm=reached[pos]
        for pos2 in data[pos][1]:
            tm2=tm+1
            if pos2 not in reached or reached[pos2]>tm2:
                reached[pos2]=tm2
                todos.add(pos2)
    return reached

def bitset_in(setv, n):
    '''is n in the bitset represented by integer setv?'''
    return (setv & (1<<n)) != 0

def bitset_add(setv, n):
    '''add n to the bitset represented by integer setv, returning the new set value'''
    return setv | (1<<n)

def do_steps1(data, mapped, time_limit):
    '''start at AA, BFS,
    data maps position to (flow rate, [tunnel destinations]),
    mapped maps position to a dict of pos and steps to get there'''
    # mapping positions to unique integers and back to use in the 'opened' bitset
    poslist=list(mapped.keys())
    posindex={ k: i for i,k in enumerate(poslist)}
    todos=set() # BFS
    startpos=('AA', 0, 0) # position and minutes elapsed and valves opened
    todos.add(startpos)
    reached={} # maps tuple of pos, opened valves to max score
    reached[ ('AA', 0) ]=0
    while len(todos)>0:
        todo=todos.pop()
        pos, tm, opened=todo
        if tm>=time_limit:
            continue
        score=reached[ (pos, opened) ]
        for pos2, steps in mapped[pos].items():
            tm2=tm+1+steps
            if tm2>time_limit or bitset_in(opened, posindex[pos2]) or data[pos2][0]<1:
                continue
            op2=bitset_add(opened, posindex[pos2])
            sc2=score+data[pos2][0]*(time_limit-tm2)
            rkey=(pos2, op2)
            if rkey not in reached or sc2>reached[rkey]:
                reached[rkey]=sc2
                todos.add( (pos2, tm2, op2) )
    return max(reached.values())

def do_steps2(data, mapped, time_limit):
    '''start at AA with elephant, BFS,
    data maps position to (flow rate, [tunnel destinations]),
    mapped maps position to a dict of pos and steps to get there'''
    # mapping positions to unique integers and back to use in the 'opened' bitset
    poslist=list(mapped.keys())
    posindex={ k: i for i,k in enumerate(poslist)}
    todos=set() # BFS
    startpos=('AA', 0, 'AA', 0, 0) # human pos., human min. elapsed, elephant pos and min. elapsed, valves opened
    todos.add(startpos)
    reached={} # maps tuple of posH, posE, opened valves to max score
    reached[ ('AA', 'AA', 0) ]=0
    while len(todos)>0:
        todo=todos.pop()
        for personi in [0, 2]: # 0 is human, 2 is elephant
            pos=todo[personi]
            tm=todo[personi+1]
            opened=todo[-1]
            if tm>=time_limit:
                continue
            score=reached[ (todo[0], todo[2], opened) ]
            for pos2, steps in mapped[pos].items():
                tm2=tm+1+steps
                if tm2>time_limit or bitset_in(opened, posindex[pos2]) or data[pos2][0]<1:
                    continue
                op2=bitset_add(opened, posindex[pos2])
                sc2=score+data[pos2][0]*(time_limit-tm2)
                newtodo=list(todo)
                newtodo[personi]=pos2
                newtodo[personi+1]=tm2
                newtodo[-1]=op2
                rkey=(newtodo[0], newtodo[2], op2)
                if rkey not in reached or sc2>reached[rkey]:
                    reached[rkey]=sc2
                    todos.add( tuple(newtodo) )
    return max(reached.values())

sample1=open('data_src/2022-day-16-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ result.group(1, 2, 3) for s in lines if (result:= re.match(r'Valve (\w+) has flow rate=(\d+); tunnel[s]* lead[s]* to valve[s]* (.*)', s)) ]
data={ tup[0]: (int(tup[1]), [s for s in tup[2].split(', ') if s]) for tup in data}
mapped={ pos: do_steps0(data, pos) for pos in data.keys() }
score1=do_steps1(data, mapped, 30)
print(f'part 1: {score1}')
score2=do_steps2(data, mapped, 26)
print(f'part 2: {score2}')

# with sample2 the scores should be 1651, 1707
# part 1: 1845
# part 2: 2286 after 14s.

In [ ]:
# 2022 day 15
# mv ~/Downloads/input* data_src/2022-day-15-input.txt
# big input file looks like: small list of big coords, including negative x
# idea: part 1 parse using re, then calc. manhattan distance, map to requested y,
#  combine ranges
# part 2: using part 1 code do a vertical sweep finding the first free horizontal position

sample2='''
Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3
'''

def get_range(row, reqy):
    '''for a single sensor/beacon row return the horizontal range at y position reqy
    where no other beacons can be'''
    sx,sy,cbx,cby=row
    mdist=abs(sx-cbx)+abs(sy-cby)
    remdist=mdist-abs(reqy-sy)
    if remdist<0:
        return None
    return (sx-remdist, sx+remdist)

def clip_range(ran1, a, b):
    '''reduce given range ran1 to fit within range a,b (or None if there is no fit)'''
    if ran1 is None:
        return None
    c,d=ran1
    if b<c or a>d:
        return None
    e=max(a, c)
    f=min(b, d)
    if e<=f:
        return (e,f)
    else:
        return None

def combine_ranges(ranges):
    '''combine ranges with overlap into one'''
    found=True # overlap found?
    while found:
        found=False
        for i, ran1 in enumerate(ranges):
            a,b=ran1
            for j,r in enumerate(ranges):
                if i>=j:
                    continue
                c,d=r
                if not (b<c or d<a): # overlap
                    ranges[i]=(min(a,c), max(b, d))
                    ranges.pop(j)
                    found=True
                    break
            if found:
                break
    # will end when no overlap is found

def count_horpositions(ranges, data, reqy):
    '''count horizontal positions in the specified ranges, but exclude any existing
    beacons at that reqy y position'''
    # list beacons at reqy
    beaconx=set()
    for row in data:
        sx,sy,cbx,cby=row
        if cby==reqy:
            beaconx.add(cbx)
    count=0
    for row in ranges:
        a,b=row
        count+=b-a+1
        # now deduct beacons in that range
        for bx in beaconx:
            if a<=bx<=b:
                count-=1
    return count

def find_free_horpos(ranges, minx, maxx):
    '''given non-overlapping ranges and a minimum and maximum x value, find the
    first free position'''
    ranges.sort(key=lambda tup: tup[0])
    if len(ranges)<1 or ranges[0][0]>minx:
        return minx
    for i,r in enumerate(ranges):
        if i==0:
            continue
        rp=ranges[i-1]
        assert rp[1]<r[0]
        if rp[1]+1<r[0]:
            return rp[1]+1
    if ranges[-1][1]<maxx:
        return ranges[-1][1]+1
    return None

sample1=open('data_src/2022-day-15-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ result.group(1, 2, 3, 4) for s in lines if (result:= re.match(r'Sensor at x=([\d\-]+), y=([\d\-]+): closest beacon is at x=([\d\-]+), y=([\d\-]+)', s)) ]
data=[ [int (n) for n in row ] for row in data ]
# part 1
reqy=2000000
#reqy=10
ranges=[]
for row in data:
    ran1=get_range(row, reqy)
    if ran1 is not None:
        ranges.append(ran1)
combine_ranges(ranges)
part1count=count_horpositions(ranges, data, reqy)
print(f'part 1: {part1count}')

# part 2
minx=0
maxx=4000000
miny=0
maxy=4000000
for y2 in range(miny, maxy+1):
    ranges=[]
    for row in data:
        ran1=get_range(row, y2)
        ran1=clip_range(ran1, minx, maxx)
        if ran1 is not None:
            ranges.append(ran1)
    combine_ranges(ranges)
    x2=find_free_horpos(ranges, minx, maxx)
    if x2 is not None:
        print(f'part 2: {x2=}, {y2=}, tuning freq. {x2*4000000+y2}')

# part 1: 4582667
# part 2: 10961118625406

In [ ]:
# 2022 day 14
# mv ~/Downloads/input* data_src/2022-day-14-input.txt
# big input file looks like: big number of paths
# idea: part 1 parse by drawing, then simulate
# part 2: continue simulation until startpos is no longer free, using a virtual floor

sample2='''
498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9
'''

def sign(n):
    if n<0:
        return -1
    elif n==0:
        return 0
    else:
        return 1

def draw_lines(board, lines):
    for row in lines:
        for desti in range(1, len(row)):
            src=row[desti-1]
            dest=row[desti]
            x,y=src
            x2,y2=dest
            dx=sign(x2-x)
            dy=sign(y2-y)
            board[(x,y)]='#'
            while not (x==x2 and y==y2):
                x+=dx
                y+=dy
                board[(x,y)]='#'

def drop_sand(board, x0, y0, maxy, part):
    '''draw grain, if drops out or not placed at all return False, if placed return True'''
    assert part in {1, 2}
    if part==1:
        assert (x0,y0) not in board # startpos empty
    else: # part==2
        if (x0,y0) in board:
            return False
    x=x0
    y=y0
    while y<=maxy+5:
        if part==2 and y==maxy+1:
            board[(x,y)]='o' # resting place on virtual floor
            return True
        if (x,y+1) not in board:
            y+=1
        elif (x-1,y+1) not in board:
            y+=1
            x-=1
        elif (x+1,y+1) not in board:
            y+=1
            x+=1
        else:
            board[(x,y)]='o' # resting place
            return True
    return False # drops out the bottom

sample1=open('data_src/2022-day-14-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
lines=[s.split(' -> ') for s in lines]
lines=[ [[int(s) for s in elem.split(',')] for elem in row] for row in lines]
board={} # maps (x,y) to # or o
draw_lines(board, lines)
maxy=max([y for x,y in board.keys()])
print(f'{maxy=}')
count=0
while drop_sand(board, 500, 0, maxy, part=1):
    count+=1
print(f'part 1: {count}')
while drop_sand(board, 500, 0, maxy, part=2):
    count+=1
print(f'part 2: {count}')

# part 1: 1513
# part 2: 22646

In [ ]:
# 2022 day 13
# mv ~/Downloads/input* data_src/2022-day-13-input.txt
# big input file looks like: long list of pairs

sample2='''
[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]
'''

def in_order(group):
    assert len(group)==2
    a=eval(group[0])
    b=eval(group[1])
    return in_order2(a, b)

def in_order2(a, b):
    '''recursive compare with ternary values; False for larger, None for equal, True for smaller'''
    if isinstance(a, int) != isinstance(b, int):
        if isinstance(a, int):
            a=[a]
        if isinstance(b, int):
            b=[b]
    if isinstance(a, int) and isinstance(b, int):
        if a==b:
            return None
        else:
            return a<b
    if isinstance(a, list) and isinstance(b, list):
        for i in range(len(a)):
            if i>=len(b):
                return False
            x=in_order2(a[i], b[i])
            if x is not None:
                return x
        if len(a)==len(b):
            return None
        else:
            return True
    assert False

def in_order3(a, b):
    x=in_order2(a, b)
    if x==True:
        return -1
    elif x is None:
        return 0
    else:
        return 1

sample1=open('data_src/2022-day-13-input.txt').read()
groups=get_line_groups(sample1.splitlines(), nostrip=False)
# part 1
part1ct=0
for idx,group in enumerate(groups):
    if in_order(group):
        part1ct+=idx+1
print(f'part 1: {part1ct}')
# part 2
lines=[]
for group in groups:
    for line in group:
        lines.append(eval(line))
lines.append([[2]])
lines.append([[6]])
lines.sort(key=functools.cmp_to_key(in_order3))
pos2=None
pos6=None
for i, line in enumerate(lines):
    if line==[[2]]:
        pos2=i+1
    elif line==[[6]]:
        pos6=i+1
print(f'part 2: {pos2*pos6}')

# part 1: 4734
# part 2: 21836

In [ ]:
# 2022 day 12
# mv ~/Downloads/input* data_src/2022-day-12-input.txt
# big input file looks like: board of 41 lines
# idea: part 1 parse as board, then BFS

sample2='''
Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi
'''

def get_height(c):
    if c=='S':
        c='a'
    elif c=='E':
        c='z'
    assert 'a'<=c<='z'
    return ord(c)-ord('a')

def do_reach(board):
    '''start at S, BFS, then return steps for E'''
    # find S
    startpos=None
    for y,row in enumerate(board):
        if (x:=row.find('S'))>=0:
            startpos=(x,y)
            break
    # find E
    endpos=None
    for y,row in enumerate(board):
        if (x:=row.find('E'))>=0:
            endpos=(x,y)
            break
    print(f'{startpos=}')
    print(f'{endpos=}')
    # BFS
    todos=set()
    todos.add(startpos)
    reached={} # maps pos to min. steps
    reached[startpos]=0
    while len(todos)>0:
        todo=todos.pop()
        x,y=todo
        ht=get_height(board[y][x])
        stps=reached[todo]
        for a,b in [(x+1, y), (x-1, y), (x, y-1), (x, y+1)]:
            if a<0 or b<0 or a>=len(board[0]) or b>=len(board):
                continue
            ht2=get_height(board[b][a])
            if ht2>ht+1:
                continue
            newpos=(a,b)
            if (newpos not in reached) or reached[newpos]>stps+1:
                reached[newpos]=stps+1
                todos.add(newpos)
    return reached[endpos]

def do_reach2(board):
    '''start at E, BFS of reverse steps, then from all board positions at elevation a return the
    minimum steps'''
    # find E as startpos
    startpos=None
    for y,row in enumerate(board):
        if (x:=row.find('E'))>=0:
            startpos=(x,y)
            break
    print(f'{startpos=}')
    # BFS
    todos=set()
    todos.add(startpos)
    reached={} # maps pos to min. steps
    reached[startpos]=0
    while len(todos)>0:
        todo=todos.pop()
        x,y=todo
        ht=get_height(board[y][x])
        stps=reached[todo]
        for a,b in [(x+1, y), (x-1, y), (x, y-1), (x, y+1)]:
            if a<0 or b<0 or a>=len(board[0]) or b>=len(board):
                continue
            ht2=get_height(board[b][a])
            if ht2<ht-1: # reversed step
                continue
            newpos=(a,b)
            if (newpos not in reached) or reached[newpos]>stps+1:
                reached[newpos]=stps+1
                todos.add(newpos)
    minsteps=[ v for k,v in reached.items() if board[k[1]][k[0]] in {'a', 'S'} ]
    minsteps.sort()
    return minsteps[0]

sample1=open('data_src/2022-day-12-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
steps1=do_reach(lines)
print(f'part 1: {steps1}')
steps2=do_reach2(lines)
print(f'part 2: {steps2}')

#part 1: 490
#part 2: 488

In [ ]:
# 2022 day 11
# mv ~/Downloads/input* data_src/2022-day-11-input.txt
# big input file looks like: monkey rules
# idea: part 1 parse ..., then ...

sample2='''
Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1
'''

class Monkey:
    def __init__(self):
        self.idx=None # id
        self.itemworries=[] # list of item worry levels
        self.op=None # '+' or '*'
        self.opval=None # integer number or 'old'
        self.testval=None # 'divisible by' integer number
        self.truem=None # true monkey number
        self.falsem=None # false monkey number
        self.inspectcount=0

    def parseblock(self, lines, idx):
        self.idx=idx
        s=lines[0].split()
        assert s[0]=='Monkey'
        assert s[1]==str(idx)+':'
        prefix='Starting items: '
        assert lines[1].startswith(prefix)
        for s in re.split(f'[\s,]', lines[1][len(prefix):]):
            if s:
                self.itemworries.append(int(s))
        s=lines[2].split()
        assert s[-3]=='old'
        self.op=s[-2]
        self.opval=s[-1]
        if self.opval!='old':
            self.opval=int(self.opval)
        s=lines[3].split()
        assert s[-2]=='by'
        self.testval=int(s[-1])
        s=lines[4].split()
        assert s[-2]=='monkey'
        self.truem=int(s[-1])
        s=lines[5].split()
        assert s[-2]=='monkey'
        self.falsem=int(s[-1])

    def __repr__(self):
        return f'Monkey {self.idx} (worries: {self.itemworries}, op: {self.op}, opval: {self.opval}, testval: {self.testval}, truem: {self.truem}, falsem: {self.falsem})'

    def do_turn(self, monks, part, modval):
        while len(self.itemworries)>0:
            w=self.itemworries.pop(0)
            opval=w if self.opval=='old' else self.opval
            if self.op=='+':
                w+=opval
            elif self.op=='*':
                w*=opval
            else:
                assert False
            if part==1:
                w//=3
            elif part==2:
                w%=modval
            else:
                assert False
            if w%self.testval==0:
                monks[self.truem].itemworries.append(w)
            else:
                monks[self.falsem].itemworries.append(w)
            self.inspectcount+=1

def do_round(monks, part, modval):
    for m in monks:
        m.do_turn(monks, part, modval)

def lcm(a, b): # least common multiple
    return a*b//math.gcd(a, b)

sample1=open('data_src/2022-day-11-input.txt').read()
groups=get_line_groups(sample1.splitlines(), nostrip=False)
monks=[] # list of Monkey
for i, lines in enumerate(groups):
    m=Monkey()
    m.parseblock(lines, i)
    monks.append(m)
# part 1
for _ in range(20):
    do_round(monks, 1, 0)
inspects=[ m.inspectcount for m in monks ]
inspects.sort()
print(f'part 1: {inspects[-2]*inspects[-1]}')
# part 2
monks=[] # list of Monkey
for i, lines in enumerate(groups):
    m=Monkey()
    m.parseblock(lines, i)
    monks.append(m)
modval=1
for m in monks:
    modval=lcm(modval, m.testval)
print(f'part 2: {modval=}')
for _ in range(10000):
    do_round(monks, 2, modval)
inspects=[ m.inspectcount for m in monks ]
inspects.sort()
print(f'part 2: {inspects[-2]*inspects[-1]}')

# part 1: 113232
# part 2: 29703395016


In [ ]:
# TEMPLATE
# 2022 day 11
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2022-day-11-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample2='''

'''

sample1=open('data_src/2022-day-11-input.txt').read()
lines=[s for s in sample2.splitlines() if len(s)>0 ]
groups=get_line_groups(sample2.splitlines(), nostrip=False)
data=[ int(s) for s in lines[0].split(',') ]
data=[ s.split() for s in lines ]
data=[ [cmd, int(num), 0] for cmd, num in data ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed